In [ ]:
!wget -q https://raw.githubusercontent.com/openforcefield/2023-workshop-vignettes/main/colab_setup.ipynb
%run colab_setup.ipynb
!mamba install "openff-nagl-models =0.0.2" "openff-nagl = 0.2.2" -c conda-forge -yq

In [ ]:
%env INTERCHANGE_EXPERIMENTAL=1
%env LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib

Grab an externally parameterized system (from the SAMPL6 challenge, used a non-SMIRNOFF FF), and show that we can load it into Interchange and evaluate energies.

In [2]:
import urllib

url = "https://raw.githubusercontent.com/samplchallenges/SAMPL6/master/host_guest/SAMPLing/CB8-G3-0/"

urllib.request.urlretrieve(url + "OPENMM/complex.xml", "complex.xml")
urllib.request.urlretrieve(url + "PDB/complex.pdb", "complex.pdb")

('complex.pdb', <http.client.HTTPMessage at 0x10f98d660>)

In [3]:
from openff.interchange import Interchange
import openmm
import openmm.app

from openff.units.openmm import ensure_quantity

from openff.interchange.drivers import get_openmm_energies, get_summary_data

In [4]:
pdb_file = openmm.app.PDBFile("complex.pdb")

topology: openmm.app.Topology = pdb_file.topology
system: openmm.System = openmm.XmlSerializer.deserialize(open("complex.xml").read())

for index, force in enumerate(system.getForces()):
    if "Barostat" in force.__class__.__name__:
        break

system.removeForce(index)

In [5]:
import nglview

nglview.show_file("complex.pdb")

NGLWidget()

In [6]:
inputted = Interchange.from_openmm(
    topology=topology,
    system=system,
    box_vectors=ensure_quantity(topology.getPeriodicBoxVectors(), "openff"),
    positions=pdb_file.positions,
)

In [7]:
get_openmm_energies(inputted)

EnergyReport(energies={'Bond': <Quantity(170.605375, 'kilojoule / mole')>, 'Angle': <Quantity(893.205219, 'kilojoule / mole')>, 'Torsion': <Quantity(503.72436, 'kilojoule / mole')>, 'Nonbonded': <Quantity(-97697.9652, 'kilojoule / mole')>})